In [9]:
import derive
reload(derive)

fun = lambda x: (3 * x[0] + x[1]) ** 2
point = [0.1, 0.3]

print derive.grad(fun)(point)
print derive.hess(fun)(point)

[ 3.6       1.200006]
[[ 17.99994875   5.99995054]
 [  5.99995054   2.00008066]]


In [69]:
import sympy as sm
import numpy as np

D = 2
x = sm.symbols(' '.join(['x%s' % i for i in xrange(D)]))
fun = (3 * x[0] + x[1]) ** 2

def grad(fun):
    x = list(fun.free_symbols)
    D = len(x)
    fns = []
    for i in xrange(D):
        rest = range(D)
        del rest[i]
        vars = [x[j] for j in rest]
        vals = [point[j] for j in rest]
        substitutions = {var: val for (var, val) in zip(vars, vals)}
        partial = sm.diff(fun.subs(substitutions), x[i])
        fns.append(sm.lambdify(x[i], partial, 'numpy'))
    
    def eval(x):
        return [fn(x[i]) for i, fn in enumerate(fns)]
    
    return eval
    
print grad(fun)(point)

[3.6, 1.2]


In [25]:
import derive
reload(derive)

import tensorflow as tf      

x = tf.Variable(0, dtype = tf.float32)
y = tf.Variable(0, dtype = tf.float32)
f = (3 * x + y) ** 2
hess = derive.hess(f)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print hess([x, y]).eval()

[[ 18.   6.]
 [  6.   2.]]


In [211]:
import theano
import theano.tensor as T

x = T.dvector('x')
y = x ** 2
cost = y.sum()
gy = T.grad(cost, x)
H, updates = theano.scan(lambda i, gy,x : T.grad(gy[i], x), sequences=T.arange(gy.shape[0]), non_sequences=[gy, x])
f = theano.function([x], H, updates=updates)